In [106]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from google.cloud import storage

In [107]:
spark = SparkSession.builder.appName('Ops').config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog").getOrCreate()

In [114]:
storage_client = storage.Client()
blobs = storage_client.list_blobs('data-lake-jlr-bronze-zone', prefix='dev/data/')

In [115]:
buckets = []
for blob in blobs:
    if not blob.name.endswith('/'):
        buckets.append(blob.name)

In [116]:
buckets

['dev/data/documentedname.csv',
 'dev/data/endcustomerlocation.csv',
 'dev/data/feature.csv',
 'dev/data/featuregroup.csv',
 'dev/data/legalentity.csv',
 'dev/data/location.csv',
 'dev/data/model.csv',
 'dev/data/news.csv',
 'dev/data/order.csv',
 'dev/data/product_l0.csv',
 'dev/data/product_l1.csv',
 'dev/data/product_l2.csv',
 'dev/data/sanctionmeasure.csv',
 'dev/data/sanctionparty.csv',
 'dev/data/sanctions.csv',
 'dev/data/sob.csv',
 'dev/data/supplier.csv']

In [45]:
for bucket in buckets:
    df = spark.read.option('header', True).option('inferSchema', True).csv(f"gs://data-lake-jlr-bronze-zone/{bucket}")
    df.write.mode("append").format("delta").save(f"gs://data-lake-jlr-gold-zone/{bucket[:-4]}")
    spark.read.format('delta').load(f'gs://data-lake-jlr-gold-zone/{bucket[:-4]}').show()

+-----+--------------------+
|docID|                name|
+-----+--------------------+
|    1|ANHUI LAND GROUP ...|
+-----+--------------------+



+----+--------------------+----------+-----------+--------------------+-----------------------------------------+
|clID|             address|  latitude|  longitude|              latlon|END_CUSTOMER_LOCATION_LOCATED_AT_LOCATION|
+----+--------------------+----------+-----------+--------------------+-----------------------------------------+
|   1|"""1-chōme-1-6 Uc...| 35.671304| 139.756642|POINT(35.671304, ...|                                      [1]|
|   2|"""Graflinger Str...|  48.84453|    12.9616|POINT(48.84453, 1...|                                      [4]|
|   3|"""4-12 Kuwait St...| 25.250603|  55.291427|POINT(25.250603, ...|                                      [7]|
|   4|"""Erkel Gyula u....|   1046"""|  47.565555|           19.092781|                     POINT(47.565555, ...|
|   5|"""Tower Bridge A...|-34.352917| -58.796389|POINT(-34.352917,...|                                     [13]|
|   6|   """Golden Gate"""|   51.5081|    -0.0759|POINT(51.508100, ...|                 

+-------+-------------+--------------------+--------------------+--------------------+-------------------------------+
|  priPN|       family|                name|           isDefault|             modelID|FEATURE_FEATURE_PRODUCT_PRODUCT|
+-------+-------------+--------------------+--------------------+--------------------+-------------------------------+
|23-4858|         null|P530 AWD Automati...|                True|                   0|                    ['23-4858']|
|67-1565|         null|P440e AWD AUTOMAT...|                True|                   1|                    ['67-1565']|
|88-5132|         null|    2.0L P250 i4 GAS|                True|                   2|                    ['88-5132']|
|18-3869|   Roof Color|   Body-colored roof|                True|                   0|                    ['18-3869']|
|61-9506|   Roof Color| Black contrast roof|               False|                   0|                    ['61-9506']|
|27-7989|    Roof Type|Sliding panoramic...|    

+---+------------+--------+----------------+---------------------------------+
|fID|isAdditional|category|            name|FEATURE_GROUP_HAS_FEATURE_FEATURE|
+---+------------+--------+----------------+---------------------------------+
|  0|       false|   Motor|          Engine|             ['23-4858', '67-1...|
|  1|       false|Exterior|            Roof|             ['18-3869', '61-9...|
|  2|       false|Exterior|          Wheels|             ['60-4613', '25-3...|
|  3|       false| Options|           Glass|             ['54-5943', '25-4...|
|  4|       false|Interior|         Seating|             ['70-1797', '38-5...|
|  5|       false|Interior| Steering wheels|             ['25-8556', '98-8...|
|  6|        true| Options|Driver Assitance|                      ['32-1165']|
+---+------------+--------+----------------+---------------------------------+



+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|leID|           legalName|         description|               mfgPN|             address|            latitude|           longitude|              region|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|   0|"""Jaguar Land Ro...|"""Jaguar Land Ro...| deal with curren...|          97426-7143|       """abbey road|             whitley|            coventry|
|   1|"""Anand Automoti...|"""ANAND is a glo...| SUJÁN. A dedicat...| places ANAND amo...|          52342-2164|"""1 Sri Aurobind...|           Hauz Khas|
|   2|"""Anhui Zhongdin...|             Ltd."""|"""Zhongding Grou...| the headquarter ...|             Ningguo| in the Southeast...| Zhongding Group ...|
|   3|"""Grupo Antolin-...|             S.A."""|"""Global supplie...|       

...

+------------+----------+-------+--------------------+--------+----+-------------+-----+-----------------------------+---------------------------------------+
|         SKU|     mfgPN|  priPN|                name|imageURI|risk|unitOfMeasure|level|PRODUCT_L2_STORED_AT_SUPPLIER|PRODUCT_L2_STORED_AT_SUPPLIER_INVENTORY|
+------------+----------+-------+--------------------+--------+----+-------------+-----+-----------------------------+---------------------------------------+
|6ISXSPXN9WH0|55025-8693|50-5469| Cylinder Head Cover|       -|   0|           ml|    2|                         [18]|                                 [1304]|
|3ZD97XDKRHTK|32250-9965|20-8025|         Piston Ring|       -|   0|           ml|    2|                         [18]|                                 [1232]|
|6XO9AU9E1030|66756-1310|52-4582|     Intake Manifold|       -|   0|           ml|    2|                         [18]|                                 [1385]|
|UOBRBFLMYJOA|21195-5661|43-8155|          Fue

+----------+-------+------------+---------------------------------------+
|sanctionID|comment|sanctionType|SANCTION_MEASURE_AFFECTS_SANCTION_PARTY|
+----------+-------+------------+---------------------------------------+
|    145224|  NPWMD|     Program|                                [19249]|
|    145256|   IFSR|     Program|                                [19249]|
|     11247|   null|       Block|                                [19249]|
+----------+-------+------------+---------------------------------------+



+--------+--------------------+-----------------------------------+----------------------------------------+---------------------------------------+
|fixedRef|            mainName|SANCTION_PARTY_AFFECTS_LEGAL_ENTITY|SANCTION_PARTY_AFFECTS_LEGAL_ENTITY_TYPE|SANCTION_PARTY_HAS_NAME_DOCUMENTED_NAME|
+--------+--------------------+-----------------------------------+----------------------------------------+---------------------------------------+
|   19249|ANHUI LAND GROUP ...|                                [2]|                                     [1]|                                    [1]|
+--------+--------------------+-----------------------------------+----------------------------------------+---------------------------------------+

+--------+---+------+-----------------------------+
|fixedRef| ID|idType|SANCTIONS_AFFECT_LEGAL_ENTITY|
+--------+---+------+-----------------------------+
|   19249|[2]|   [1]|                          [2]|
+--------+---+------+-------------------------

+-----+----------+-----------------+--------------------+----------+----------+---------+--------+-----------+--------------------------+------------------------+----------------------+---------------------------------+
|sobID|isStandard|       familyName|           modelName| orderDate|targetDate|unitPrice|quantity|totalAmount|SOB_ORDERS_FEATURE_FEATURE|SOB_ROOT_PRODUCT_PRODUCT|SOB_ORDERS_MODEL_MODEL|SOB_TARGETS_END_CUSTOMER_LOCATION|
+-----+----------+-----------------+--------------------+----------+----------+---------+--------+-----------+--------------------------+------------------------+----------------------+---------------------------------+
|    1|      true|   Range Rover SE|Range Rover SE P4...|2022-12-19|2023-01-22|   204875|       2|     456650|      ['23-4858', '18-3...|             ['19-9708']|                   [0]|                              [1]|
|    2|     false|   Range Rover SE|Range Rover SE P4...|2022-12-30|2023-02-15|   230000|       1|     230000|      ['23

+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------------------+----------------------------+
|eID|                name|         description|         eIdentifier|             address|            latitude|           longitude|              latLon|            Products|                risk|            capacity|SUPPLIER_BELONGS_TO_LEGAL_ENTITY|SUPPLIER_LOCATED_AT_LOCATION|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------------------+----------------------------+
|  0|"""vehicle manufa...|               nitra|         slovakia"""|"""The state-of-t...| 74 acre manufact...| sourced exclusiv...| and sends zero w...| staff recycle

In [50]:
df = spark.read.format('delta').load('gs://data-lake-jlr-gold-zone/dev/data/product_l1')

In [54]:
df.select('SKU',
 'mfgPN',
 'priPN',
 'name',
 'imageURI',
 'risk',
 'unitOfMeasure',
 'level').show()

+------------+----------+-------+--------------------+--------------------+--------------------+-------------+-----+
|         SKU|     mfgPN|  priPN|                name|            imageURI|                risk|unitOfMeasure|level|
+------------+----------+-------+--------------------+--------------------+--------------------+-------------+-----+
|3Y2WW7IZV7C8|26691-4744|23-4858|P530 AWD Automati...|                   -|                   0|        piece|    1|
|3VVJZR5HUMVU|50471-6503|67-1565|P440e AWD AUTOMAT...|                   -|                   0|        piece|    1|
|4VDDFH7Q9DBM|24551-9919|88-5132|    2.0L P250 i4 GAS|                   -|                   0|        piece|    1|
|77NF8PVNJD6I|55159-5618|18-3869|   Body-colored roof|                   -|                   0|        piece|    1|
|7J0QNASWCTGD|48346-8476|61-9506| Black contrast roof|                   -|                   0|        piece|    1|
|YE9MOUMBCG9V|23136-1088|27-7989|Sliding panoramic...|          